In [ ]:
import transcriptic 
from transcriptic import commands, Container
from transcriptic.config import Connection
from transcriptic.jupyter import objects
import json
from pysd2cat.data import tx_fcs
from pysd2cat.data import pipeline
from pysd2cat.analysis import biofab_live_dead_analysis as blda

from os.path import expanduser
import requests
import os
import zipfile
import shutil
import fnmatch
import glob
import pandas as pd
import ast

%load_ext autoreload
%reload_ext autoreload
%autoreload 2



#run_id='r1dd37mcxv5pf4'
run_id='r1dk8xp9dymm54'
work_dir='data/transcriptic/'+run_id
Connection.from_file("~/.transcriptic")
tx_config = json.load(open(os.path.join(expanduser("~"), ".transcriptic")))
transcriptic_email = tx_config['email']
transcriptic_token = tx_config['token']



if not os.path.exists(work_dir):
    os.makedirs(work_dir)

In [ ]:
from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

In [ ]:
run_obj = tx_fcs.get_tx_run(run_id)
d = run_obj.data


In [76]:
d.loc[d['Name'].str.contains('Flow \(Sytox\) Plate 1')]

,Name,DataType,Operation,AnalysisTool,Datasets
4,Flow (Sytox) Plate 1_0.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x131248b00>
6,Flow (Sytox) Plate 10_50.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x131248860>
7,Flow (Sytox) Plate 11_60.5 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x129e5b2b0>
8,Flow (Sytox) Plate 12_72.0 hour,file,flow_analyze,None,<transcriptic.jupyter.objects.Dataset object at 0x12b3fb780>


In [ ]:
def extract_time_point(x):
    return x.Name.split(' ')[3].split('_')[0]
def extract_time(x):
    return x.Name.split(' ')[3].split('_')[1]
def assign_container(prefix, x, containers):
    name = prefix + ' ' + x.time_point
    #print(containers)
    return next(c for c in containers.Containers if name in c.attributes['label'])


try:
    containers = run_obj.containers
except Exception as e:
    containers = run_obj.containers


flow_data = d.loc[d.Operation == 'flow_analyze']
flow_data.loc[:,'time_point'] = flow_data.apply(extract_time_point, axis=1)
flow_data.loc[:,'time'] = flow_data.apply(extract_time, axis=1)
flow_data.loc[:, 'container'] = flow_data.apply(lambda x: assign_container('Flow (Sytox) Plate', x, containers), axis=1)
flow_data.loc[:, 'container_name'] = flow_data.apply(lambda x: x['container'].attributes['label'], axis=1)



flow_data

In [ ]:
flow_data.container[4].attributes

In [ ]:
def get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=False, container="Flow (Sytox) Plate 1"):
    print("Getting data for container: " + str(container))
    fcs_path = os.path.join(work_dir, container.replace(" ", "_"))
    if not os.path.exists(fcs_path):
        os.makedirs(fcs_path)
    file_info = tx_fcs.create_fcs_manifest_and_get_files(run_id, 
                                                         transcriptic_email, 
                                                         transcriptic_token,
                                                         fcs_path=fcs_path, 
                                                         download_zip=download_data,
                                                         container_name=container
                                                         )
    print(file_info)
    #Flow plate didn't have the properties, so need to get them from another plate
    inoc_plate_info = tx_fcs.get_plate_well_properties(tx_fcs.get_tx_run(run_id), 
                                                       container_name="Inoculation Plate")
    file_info['container'] = container

    for well_id, info in file_info['aliquots'].items():
        info['properties'] = inoc_plate_info[well_id.upper()]
    return file_info

In [ ]:


def make_row(well, info, timeseries_map=False):
    row = { 
        "well" : well,
#        "filename" : info['file'],
#        "checksum" : info['checksum']
        }
    if 'file' in info:
        row['filename'] = info['file']
    if 'checksum' in info:
        row['checksum'] = info['checksum']
    # Properties don't include sytox, so add manually
    def well_stain(well):
        if 'a' in well or 'b' in well or 'c' in well or 'd' in well:
            return "SYTOX Red Stain"
        else:
            return None
        
    row['stain'] = well_stain(well) 
    
    def expand_options(v):
        #print(v)
        kv_pairs = {}
        for option, value in v.items():
            if option == 'Reagents':
                #print(option)
                for reagent, properties in value.items():
                    if reagent == 'Ethanol':
                        kv_pairs['kill'] = reagent
                        kv_pairs['kill_volume'] = properties['final_concentration']['qty']
        return kv_pairs
    
    if timeseries_map:
        row['kill'] = 'Ethanol'
        killed = '9' not in well
        row['kill_volume'] = 300.0 if killed else 0.0 
    else:
        if 'properties' in info:
            for k, v in info['properties'].items():
                if k == 'Options':
                    row.update(expand_options(ast.literal_eval(v)))
                else:
                    row[k] = str(v)
                
                
                
    return row

def make_meta_dataframe(run_id, plate_properties, timeseries_map=False):
    data_df = pd.DataFrame()
    for well, info in plate_properties['aliquots'].items():
        row = make_row(well, info, timeseries_map=timeseries_map)
        data_df = data_df.append(row, ignore_index=True)
        
    data_df['experiment_id'] = run_id
    return data_df
        
def get_container_work_dir(out_dir, container):
    return os.path.join(out_dir, container.replace(" ", "_"))

def get_container_data(out_dir, container):
    path = os.path.join(get_container_work_dir(out_dir, container), 'data.csv')
    df = pd.read_csv(path)
    return df

def make_container_dataframes(run_id, plate_properties_list, out_dir, timeseries_map=False, overwrite=False):
    for plate_properties in plate_properties_list:
        df_path = get_container_work_dir(out_dir, plate_properties['container'])
        if not os.path.exists(df_path):
            os.makedirs(df_path)
        df_file = os.path.join(df_path, 'data.csv')
        if overwrite or not os.path.exists(df_file):
            meta_df = make_meta_dataframe(run_id, plate_properties, timeseries_map=timeseries_map)
            df = pipeline.get_data_and_metadata_df(meta_df, '.', fraction=None, max_records=30000)
            # drop bead controls in column 12
            df = df.loc[~df['well'].str.contains('12')]
            df.to_csv(df_file)

In [66]:
[flow_data.container[i].attributes['label'] for i in flow_data.index]



['Flow (Sytox) Plate 1',
 'Flow (Sytox) Plate 2',
 'Flow (Sytox) Plate 10',
 'Flow (Sytox) Plate 11',
 'Flow (Sytox) Plate 12',
 'Flow (Sytox) Plate 3',
 'Flow (Sytox) Plate 4',
 'Flow (Sytox) Plate 5',
 'Flow (Sytox) Plate 6',
 'Flow (Sytox) Plate 7',
 'Flow (Sytox) Plate 8',
 'Flow (Sytox) Plate 9']

In [78]:
plate_properties = [get_flow_plate_properties(run_id, transcriptic_email, transcriptic_token, work_dir, download_data=True, container=flow_data.container[i].attributes['label']) for i in flow_data.index]
make_container_dataframes(run_id, plate_properties, work_dir, timeseries_map=True, overwrite=True)

Getting data for container: Flow (Sytox) Plate 1
Attempting to fetch 16 datasets...


02:39:51 INFO:[{'name': 'T1.zip', 'bucket': 'transcriptic-uploads', 'key': 'uploads/6e826d72-bc20-420d-a65d-e9a5dab75b38/T1.zip'}]


                               Name     DataType     Operation AnalysisTool                                                      Datasets
0   FLUOR-post-inoculation-read      platereader  fluorescence  None         <transcriptic.jupyter.objects.Dataset object at 0x600a2aeb8>
1   OD-post-inoculation-read         platereader  absorbance    None         <transcriptic.jupyter.objects.Dataset object at 0x229cb8dd8>
2   OD-0.0 hour                      platereader  absorbance    None         <transcriptic.jupyter.objects.Dataset object at 0x12b9c3f28>
3   FLUOR-0.0 hour                   platereader  fluorescence  None         <transcriptic.jupyter.objects.Dataset object at 0x127eeef28>
4   Flow (Sytox) Plate 1_0.5 hour    file         flow_analyze  None         <transcriptic.jupyter.objects.Dataset object at 0x127eee7b8>
5   Flow (Sytox) Plate 2_2.0 hour    file         flow_analyze  None         <transcriptic.jupyter.objects.Dataset object at 0x12c9adcc0>
6   Flow (Sytox) Plate 10_50.0 hou

02:40:27 INFO:Requesting Data from: https://secure.transcriptic.com/upload/url_for?key=uploads/6e826d72-bc20-420d-a65d-e9a5dab75b38/T1.zip
02:40:27 INFO:Writing data to: data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs.zip


{'aliquots': {'a7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A7.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': '2a82dd9f1492317166ee0827a8d61fb17d3c8321'}, 'a6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_A6.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c9f262879feb2d6c8497ba51d5cc64c3b1983c1d'}, 'b8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B8.fcs', 'properties': {'Stain': 'Sytox'}, 'checksum': 'c3fc17b08ef11f0ddbe4000a38e6650286c4e319'}, 'b9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_1/fcs/CellDeath-EtOH-09-19-2019-T1_Sytox-YeastGates_SytoxYG-Plate_B9.fcs', 'properties': {}, 'checksum': '7e0c7f13c9b2ef1c1a900def881b7c74531e1d5a'}, 'a5': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plat

...

Getting data for container: Flow (Sytox) Plate 9
Attempting to fetch 16 datasets...


02:47:22 INFO:[{'name': 'T9.zip', 'bucket': 'transcriptic-uploads', 'key': 'uploads/d8403d11-073d-4a64-89d9-35dc9036853d/T9.zip'}]


                               Name     DataType     Operation AnalysisTool                                                      Datasets
0   FLUOR-post-inoculation-read      platereader  fluorescence  None         <transcriptic.jupyter.objects.Dataset object at 0x1182c1fd0>
1   OD-post-inoculation-read         platereader  absorbance    None         <transcriptic.jupyter.objects.Dataset object at 0x135bb0320>
2   OD-0.0 hour                      platereader  absorbance    None         <transcriptic.jupyter.objects.Dataset object at 0x3bbf09e48>
3   FLUOR-0.0 hour                   platereader  fluorescence  None         <transcriptic.jupyter.objects.Dataset object at 0x1fe604b00>
4   Flow (Sytox) Plate 1_0.5 hour    file         flow_analyze  None         <transcriptic.jupyter.objects.Dataset object at 0x1377c5908>
5   Flow (Sytox) Plate 2_2.0 hour    file         flow_analyze  None         <transcriptic.jupyter.objects.Dataset object at 0x12958edd8>
6   Flow (Sytox) Plate 10_50.0 hou

02:47:33 INFO:Requesting Data from: https://secure.transcriptic.com/upload/url_for?key=uploads/d8403d11-073d-4a64-89d9-35dc9036853d/T9.zip
02:47:33 INFO:Writing data to: data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs.zip


{'aliquots': {'e7': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T9_Sytox-YeastGates_SytoxYG-Plate_E7.fcs', 'properties': {}, 'checksum': '6cb00b0c8911b94f2544fcc09ea7bb01d952f56f'}, 'e6': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T9_Sytox-YeastGates_SytoxYG-Plate_E6.fcs', 'properties': {}, 'checksum': '61cffaa53c4ca63d2f0fc60d1045b25afe52b67d'}, 'f8': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T9_Sytox-YeastGates_SytoxYG-Plate_F8.fcs', 'properties': {}, 'checksum': '04377322bd04f3128f622339cf7505b624d2209d'}, 'f9': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T9_Sytox-YeastGates_SytoxYG-Plate_F9.fcs', 'properties': {}, 'checksum': '28d6c25c7077084e9c9ed47ba9e833f25a9ade96'}, 'e5': {'file': 'data/transcriptic/r1dk8xp9dymm54/Flow_(Sytox)_Plate_9/fcs/CellDeath-EtOH-09-19-2019-T9_Sytox-Yeast

In [80]:
for p in plate_properties:
    df = get_container_data(work_dir, p['container'])
    time_point = flow_data.loc[flow_data['container_name'] == p['container']]['time_point'].iloc[0]
    dead_volumes = list(df.kill_volume.dropna().unique())
    dead_volumes.remove(0.0)
    #print(dead_volumes)
    blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            dead_volumes=dead_volumes,
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                           time_point=time_point)



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 300.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
data_columns: ['Unnamed: 0', 'checksum', 'filename', 'kill', 'kill_volume', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


03:05:19 DEBUG:Splitting Test Harness Data ...


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


03:05:19 DEBUG:Running Test Harness ...


...


HI
----------------------------------------------------------------------------------------------------
Starting run at time 20:51:45
Normalizing training and testing splits...
Starting Classification training...
Training time was: 137.48 seconds
Testing time was: 3.03 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  aPOGMG15BlqMm  2019-09-25  20:51:45  random_forest_classification  Hamed        2           0.894     0.895              0.963      0.874              

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
08:54:24 DEBUG:Splitting Tes

['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


08:54:25 DEBUG:Running Test Harness ...



HI
----------------------------------------------------------------------------------------------------
Starting run at time 20:54:25
Normalizing training and testing splits...
Starting Classification training...
Training time was: 135.71 seconds
Testing time was: 2.43 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5dEljPzrdr3YG  2019-09-25  20:54:25  random_forest_classification  Hamed        2           0.937     0.944              0.986      0.947

In [84]:
for p in plate_properties:
    df = get_container_data(work_dir, p['container'])
    time_point = flow_data.loc[flow_data['container_name'] == p['container']]['time_point'].iloc[0]
    dead_volumes = list(df.kill_volume.dropna().unique())
    dead_volumes.remove(0.0)

    pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                               dead_strain_name=max(dead_volumes),
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live',
                                              time_point=time_point)
    out_file = os.path.join(get_container_work_dir(work_dir, p['container']), 'data.csv')
    pred_df.to_csv(out_file)



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


output_col: live
SYTOX Red Stain
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0, 'dead_volume': 300.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'time_point': '1'}
data_columns: ['Unnamed: 0', 'checksum', 'filename', 'kill', 'kill_volume', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
02:13:55 DEBUG:Splitting Test Harness Data ...
02:13:56 DEBUG:Running Test Harness ...


...


----------------------------------------------------------------------------------------------------
Starting run at time 15:36:23
Normalizing training and testing splits...
Starting Classification training...
Training time was: 138.35 seconds
Testing time was: 2.98 seconds
Prediction time of untested data was: 14.873516082763672

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QM28ABgwyNWx9  2019-09-26  15:36:23  random_forest_classification

03:39:09 DEBUG:Extracting Test Harness Predictions ...


Run finished at 15:39:09. Total run time = 165.76 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^






03:39:09 DEBUG:Selecting run: Run ID                         QM28ABgwyNWx9                                                                                                                                                    
Date                           2019-09-26                                                                                                                                                       
Time                           15:36:23                                                                                                                                                         
Model Name                     random_forest_classification                                                                                                                                     
Model Author                   Hamed                                                                                                                                                            
Num C

output_col: live


In [36]:
df.kill_volume.value_counts()

29.0     240000
170.0    240000
105.0    240000
64.0     240000
0.0      240000
Name: kill_volume, dtype: int64

In [44]:
# Multiple time points and 0/15 eth
blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            dead_volumes=[29.0, 64.0, 170.0, 105.0],
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                           time_point=time_point)



{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 29.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 29.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 64.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 64.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 170.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 170.0, 'stain': nan, 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 105.0, 'stain': 'SYTOX Red Stain', 'time_point': '1'}
{'experiment_id': 'r1dk8xp9dymm54', 'r

In [64]:
# Single time point and Eth titration
blda.train_models_for_stats(df, 
                            data_dir=work_dir, 
                            fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'])


{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain'}
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0.0, 'dead_volume': 980.0, 'stain': None}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 22:31:18
Normalizing training and testing splits...
Starting Classification training...
Training time was: 40.89 seconds
Testing time was: 0.64 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                       Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  QgDA8YOg58WPP  2019-07-28  22:31:18  random_forest_classification  Hamed        2           0.994     0.994              1.0        1.0                0.994     0.993     

...

HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:35:41
Normalizing training and testing splits...
Starting Classification training...
Training time was: 62.62 seconds
Testing time was: 1.57 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5ZyJ9Xak7e2xw  2019-07-28  23:35:41  random_forest_classification  Hamed        2           0.647     0.647              0.709      0.677              0.673     

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


HI
----------------------------------------------------------------------------------------------------
Starting run at time 23:36:53
Normalizing training and testing splits...
Starting Classification training...
Training time was: 52.32 seconds
Testing time was: 1.77 seconds

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                    Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  52LjE9Q3LRJMM  2019-07-28  23:36:53  random_forest_classification  Hamed        2           0.638     0.638              0.691      0.673              0.64      0.637      0.

In [112]:
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)

pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 
                                      'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W',
                                      'BL1-W', 'RL1-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          output_col = 'live')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:03:08
Normalizing training and testing splits...
Starting Classification training...
Training time was: 41.69 seconds
Testing time was: 0.75 seconds
Prediction time of untested data was: 16.60473394393921

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                      Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5W3jAjw9ez6zr  2019-07-30  14:03:08  random_forest_classification  Hamed        2      

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W']



/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


----------------------------------------------------------------------------------------------------
Starting run at time 14:04:30
Normalizing training and testing splits...
Starting Classification training...
Training time was: 45.57 seconds
Testing time was: 0.85 seconds
Prediction time of untested data was: 16.545056104660034

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                         Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  EWoyeqNla8WWM  2019-07-30  14:04:30  random_forest_classification  Hamed        2           0.994  

In [116]:
# Train on just side scatter
df = pd.read_csv(os.path.join(work_dir, 'data.csv'), index_col=0)
pred_df = blda.train_models_for_prediction(df, data_dir=work_dir,
                                 fcs_columns=[ 'SSC-A','SSC-H', 'SSC-W'],
                                           combine_stains=False,
                                          overwrite=True,
                                          additional_description={
                                              'channels' : [ 'SSC-A','SSC-H', 'SSC-W']
                                          },
                                          output_col = 'live_ss')
pred_df.to_csv(os.path.join(work_dir, 'data.csv'))

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,4,5,6,7,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


SYTOX Red Stain
{'experiment_id': 'r1dd37mcxv5pf4', 'random_state': 0, 'live_volume': 0, 'dead_volume': 980.0, 'stain': 'SYTOX Red Stain', 'prediction': True, 'channels': ['SSC-A', 'SSC-H', 'SSC-W']}
data_columns: ['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live']


/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:10:11
Normalizing training and testing splits...
Starting Classification training...
Training time was: 27.55 seconds
Testing time was: 0.84 seconds
Prediction time of untested data was: 17.089905261993408

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                               Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  6gD7RVbGQPB6N  2

/Users/danbryce/Documents/sift/xplan/external/xplan/xplan-to-autoprotocol-reactor/xplan-dev-env/xplan-dev-env/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


['SSC-A', 'SSC-H', 'SSC-W']

----------------------------------------------------------------------------------------------------
Starting run at time 14:11:17
Normalizing training and testing splits...
Starting Classification training...
Training time was: 28.10 seconds
Testing time was: 1.36 seconds
Prediction time of untested data was: 21.104886054992676

          Run ID        Date      Time                    Model Name Model Author Num Classes  Accuracy  Balanced Accuracy  AUC Score  Average Precision  F1 Score  Precision  Recall Samples In Train Samples In Test                                                                                      Model Description Column Predicted Num Features Used                                                                                                                                                 Data and Split Description Normalized Num Features Normalized Feature Extraction Was Untested Data Predicted
0  5RvNyJBOeZwrM  2019-07-30  14:

In [117]:
pred_df.columns

Index(['Container Type', 'Control', 'Culture_Volume', 'Media', 'Replicate', 'Storage (C)', 'Strain', 'checksum', 'filename', 'kill', 'kill_volume', 'plate_id', 'source', 'stain', 'well', 'experiment_id', 'Time', 'FSC-A', 'SSC-A', 'BL1-A', 'RL1-A', 'FSC-H', 'SSC-H', 'BL1-H', 'RL1-H', 'FSC-W', 'SSC-W', 'BL1-W', 'RL1-W', 'index', 'live', 'live_ss'], dtype='object')

In [98]:
len(pred_df.loc[pred_df.stain == 'SYTOX Red Stain'])

1098660